In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import keras
import tensorflow as tf
from keras.applications import ResNet152V2, ResNet50V2
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, Input
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.utils import shuffle

Generate Data From chest_xray Folder:
* Here we split the data in a way that we have bacteria, virus and normal images. in the proper way- some of each and not in a total randomize way
* TODO: Need to chek why the depth of the image is 3 and not 1 as in regular cnn network

In [4]:
# IMG_SIZE = 150
IMG_DEPTH = 3
IMG_SIZE = 224
BATCH = 16
SEED = 42

normal_dataset = glob.glob('/content/drive/MyDrive/Deep Learning Project/chest_xray/NORMAL/*.jpeg')
pneumonia_dataset = glob.glob('/content/drive/MyDrive/Deep Learning Project/chest_xray/PNEUMONIA/*.jpeg')
virus_dataset = list(filter(lambda x: 'virus' in x, pneumonia_dataset))
bacterial_dataset = list(filter(lambda x: 'bacteria' in x, pneumonia_dataset))


Generate Train, Validation and Test Sets:

In [5]:
def split_data(dataSet, testSize, valSize):
    train, test= train_test_split(dataSet, test_size=testSize, random_state=SEED, shuffle=True)
    train, val= train_test_split(train, test_size=valSize, random_state=SEED, shuffle=True)
    return train, test, val


train_normal, test_normal, val_normal = split_data(normal_dataset, 0.15, 0.038)
train_bacterial, test_bacterial, val_bacterial = split_data(bacterial_dataset, 0.05, 0.0095)
train_virus, test_virus, val_virus = split_data(virus_dataset, 0.075, 0.019)



train = [x for x in train_normal]
train.extend([x for x in train_bacterial])
train.extend([x for x in train_virus])

test = [x for x in test_normal]
test.extend([x for x in test_bacterial])
test.extend([x for x in test_virus])

val = [x for x in val_normal]
val.extend([x for x in val_bacterial])
val.extend([x for x in val_virus])

df_train = pd.DataFrame(np.concatenate([['Normal']*(len(train_normal)) , ['Pneumonia']*(len(train_bacterial) + len(train_virus))]), columns = ['class'])
df_train['image'] = [x for x in train]

df_val = pd.DataFrame(np.concatenate([['Normal']*(len(val_normal)) , ['Pneumonia']*(len(val_bacterial) + len(val_virus))]), columns = ['class'])
df_val['image'] = [x for x in val]

df_test = pd.DataFrame(np.concatenate([['Normal']*len(test_normal) , ['Pneumonia']*(len(test_bacterial) + len(test_virus))]), columns = ['class'])
df_test['image'] = [x for x in test]



Perform Data Augmentation on Each Set:
* TODO: Need to check why we are doing data augmentation on all 3 sets
  and not just on the training set as in the regular cnn network.

In [6]:

# With data augmentation to prevent overfitting and handling the imbalance in dataset
# Because the dataset is small we "increase" the dataset by change of images parameters.
# In this way we increase our dataset and prevent overfitting.

train_datagen = ImageDataGenerator(rescale=1/255.,
                                  zoom_range = 0.1,
                                  #rotation_range = 0.1,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1)

test_datagen = ImageDataGenerator(rescale=1/255.)

ds_train = train_datagen.flow_from_dataframe(df_train,
                                             #directory=train_path, #dataframe contains the full paths
                                             x_col = 'image',
                                             y_col = 'class',
                                             target_size = (IMG_SIZE, IMG_SIZE),
                                             class_mode = 'binary',
                                             batch_size = BATCH,
                                             seed = SEED)

ds_val = test_datagen.flow_from_dataframe(df_val,
                                            #directory=train_path,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'binary',
                                            batch_size = BATCH,
                                            seed = SEED)

ds_test = test_datagen.flow_from_dataframe(df_test,
                                            #directory=test_path,
                                            x_col = 'image',
                                            y_col = 'class',
                                            target_size = (IMG_SIZE, IMG_SIZE),
                                            class_mode = 'binary',
                                            batch_size = 1,
                                            shuffle = False)

Found 5262 validated image filenames belonging to 2 classes.
Found 105 validated image filenames belonging to 2 classes.
Found 489 validated image filenames belonging to 2 classes.


# Building CNN Network- Transfer Learning with Fine Tuning

Create and Freeze the Base Model and Get the Pre-Trained Model
* This function creates a base model and freezeing all of its layers.

In [7]:
def create_base_model():
    # --------- create base model ----------#
    base_model = ResNet50V2(
        weights='imagenet',
        input_shape=(IMG_SIZE, IMG_SIZE, IMG_DEPTH),
        include_top=False)

    # --------- FREEZE base model ----------#
    base_model.trainable = False
    return base_model


Get the Pre-Trained Model
* If you are creating many models in a loop, this global state will consume
  an increasing amount of memory over time, and you may want to clear it.
  Calling clear_session() releases the global state: this helps avoid clutter
  from old models and layers, especially when memory is limited.
* We use: keras.backend.clear_session()

In [8]:
def get_pre_trained_model(base_model):
    # --------- get pretrained model ----------#
    keras.backend.clear_session()

    #Input shape = [width, height, color channels]
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, IMG_DEPTH))

    x = base_model(inputs, training=False)

    # Head
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    #Final Layer (Output)
    output = Dense(1, activation='sigmoid')(x)

    pre_trained_model = keras.Model(inputs=[inputs], outputs=output)

    return pre_trained_model

Adding Fine Tuning
* This function un-freezes the top layers of the base model.

In [9]:
def add_fine_tuning(base_model):
    # Set the base_model as trainable
    base_model.trainable = True

    # Print the number of layers in the base_model
    print(f"Number of Layers In the Base Model: {len(base_model.layers)}")

    # Specify the layer index from which fine-tuning will start
    fine_tune_from = -70

    # Iterate through the layers of the base_model
    # and set them as non-trainable up to the fine_tune_from index
    for layer in base_model.layers[:fine_tune_from]:
        layer.trainable = False

Try Different Epochs and Learning Rates
* Compile and Train the Model for each Epoch and Learning Rate.

* Plot Graphs of the Training and Validation Accuracy and Loss vs Epochs.

* It is critical to only do fine tuning after the model with frozen layers has been trained to convergence.
  If you mix randomly-initialized trainable layers with trainable layers that hold pre-trained features,
  the randomly-initialized layers will cause very large gradient updates during training, which will destroy your pre-trained features.

* It's also critical to use a very low learning rate at this stage, because you are training a much larger model than in the first round of training, on a dataset that is typically very small. As a result, you are at risk of overfitting very quickly if you apply large weight updates. Here, you only want to readapt the pretrained weights in an incremental way.

In [ ]:
Epochs = [40]
learning_rates= [0.01, 0.001, 0.0001]

opt = Adam

lr_epoch_test_acc = {}
lr_epoch_test_loss = {}
for Epoch in Epochs:
  initial_epochs = Epoch//2
  for lr in learning_rates:
    base_model = create_base_model()
    model = get_pre_trained_model(base_model)
    # Training the model with all layers frozen
    model.compile(loss='binary_crossentropy',
                optimizer = opt(learning_rate=lr), metrics='accuracy')
    # model.summary()
    history = model.fit(ds_train,
            batch_size=BATCH, epochs=initial_epochs,
            validation_data=ds_val,
            steps_per_epoch=(len(df_train)/BATCH),
            validation_steps=(len(df_val)/BATCH))
    # Evaluate the Accuracy and Loss (Before Fine Tuning)
    train_acc = history.history['accuracy']
    train_loss = history.history['loss']
    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']

    # adding fine tuning
    add_fine_tuning(base_model)
    model.compile(loss='binary_crossentropy',
                optimizer = opt(learning_rate=lr/10), metrics='accuracy')
    history_fine = model.fit(ds_train,
                        batch_size=BATCH,
                        epochs=Epoch,
                        initial_epoch=history.epoch[-1],
                        validation_data=ds_val,
                        steps_per_epoch=(len(df_train)/BATCH),
                        validation_steps=(len(df_val)/BATCH))
    # Evaluate the Accuracy and Loss (After Fine Tuning):

    test_loss, test_accuracy = model.evaluate(ds_test, verbose=0)
    lr_epoch_tuple = (Epoch, lr)
    lr_epoch_test_acc[lr_epoch_tuple] = test_accuracy
    lr_epoch_test_loss[lr_epoch_tuple] = test_loss
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)

    train_acc += history_fine.history['accuracy']
    val_acc += history_fine.history['val_accuracy']
    train_loss += history_fine.history['loss']
    val_loss += history_fine.history['val_loss']
    epochs = range(1, len(train_acc) + 1)

    # Create a figure and subplot
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))
    # Plot the training and validation accuracy
    axs[0].plot(train_acc, 'bo-', label='Training Accuracy')
    axs[0].plot(val_acc, 'ro-', label='Validation Accuracy')
    axs[0].set_title(f'Training & Validation Accuracy\n opt={opt.__name__}, lr={lr}, Epochs={Epoch}')
    # axs[0].title.set_size(10) # if title is too big, change the size here
    axs[0].legend(loc='lower right')
    axs[0].set_xlabel("Epochs")
    axs[0].set_ylabel("Accuracy")

    # Plot the training and validation loss
    axs[1].plot(train_loss, 'bo-', label='Training Loss')
    axs[1].plot(val_loss, 'ro-', label='Validation Loss')
    axs[1].set_title(f'Training & Validation Loss\n opt={opt.__name__}, lr={lr}, Epochs={Epoch}')
    # axs[1].title.set_size(10) # if title is too big, change the size here
    axs[1].legend(loc='upper right')
    axs[1].set_xlabel("Epochs")
    axs[1].set_ylabel("Loss")

    # Adjust spacing between subplots
    plt.subplots_adjust(wspace=0.2)
    plt.show()


Epoch 1/20
328/328 [==============================] - 132s 391ms/step - loss: 0.3112 - accuracy: 0.9105 - val_loss: 0.1957 - val_accuracy: 0.9048
Epoch 2/20
328/328 [==============================] - 127s 387ms/step - loss: 0.1578 - accuracy: 0.9445 - val_loss: 0.1151 - val_accuracy: 0.9619
Epoch 3/20
328/328 [==============================] - 130s 397ms/step - loss: 0.1552 - accuracy: 0.9434 - val_loss: 0.1366 - val_accuracy: 0.9714
Epoch 4/20
328/328 [==============================] - 127s 387ms/step - loss: 0.1528 - accuracy: 0.9432 - val_loss: 0.1182 - val_accuracy: 0.9524
Epoch 5/20
328/328 [==============================] - 126s 382ms/step - loss: 0.1580 - accuracy: 0.9394 - val_loss: 0.0919 - val_accuracy: 0.9619
Epoch 6/20
328/328 [==============================] - 126s 382ms/step - loss: 0.1435 - accuracy: 0.9500 - val_loss: 0.0904 - val_accuracy: 0.9714
Epoch 7/20
328/328 [==============================] - 126s 383ms/step - loss: 0.1403 - accuracy: 0.9472 - val_loss: 0.0845 -

Print the Loss & Accuracy Dictionaries

In [ ]:
print(f"Test Accuracy and Loss Results with Optimizer: {opt.__name__}")
print()
print()
def print_results_dict(Accuracy, Loss):
    print("Print Accuracy Results")
    for key, val in Accuracy.items():
        epochs, lr = key
        print(f"Epochs Number = {epochs}, Learning Rate = {lr}, Accuracy = {val:.3f}")
    print()
    print()
    print("Print Loss Results")
    for key, val in Loss.items():
        epochs, lr = key
        print(f"Epochs Number = {epochs}, Learning Rate = {lr}, Loss = {val:.3f}")


print_results_dict(lr_epoch_test_acc, lr_epoch_test_loss)